In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
import re

In [93]:
def get_url_list():
    with open("./wd_link") as f:
        urls = f.readlines()
    
    urls = [url.rstrip('\n') for url in urls]
    
    return urls

def get_job_description(driver, url):
    driver.get(url)
    time.sleep(3)
    
    res = {}
    
    # JobHeader 클래스 가져와서 title, company 추출
    job_header = driver.find_element(By.CLASS_NAME, 'JobHeader_className__HttDA')
    title = job_header.find_element(By.TAG_NAME, 'h2')
    company = job_header.find_element(By.TAG_NAME, 'a')
    
    # res 딕셔너리에 저장
    res['title'] = title.text
    res['company'] = company.text

    # JobDescription 클래스 가져와서 header, contents 추출
    jd = driver.find_element(By.CLASS_NAME, 'JobDescription_JobDescription__VWfcb')
    jd_header = jd.find_elements(By.TAG_NAME, 'h6')
    jd_contents = jd.find_elements(By.TAG_NAME, 'p')

    for i in range(1, len(jd_header)+1):
        title = jd_header[i-1].text
        content = jd_contents[i].text
    
        if title == '자격요건':
            res['qualifications'] = content
        elif title == '우대사항':
            res['preferred'] = content
            break

        elif title == '기술스택 ・ 툴':
            skill_list = content.split("\n")
            res["skills"] = skill_list
        
    return res

In [96]:
driver = webdriver.Chrome()

# url = 'https://www.wanted.co.kr/wd/164241'

urls = get_url_list()
total = []

for url in urls:
    res = get_job_description(driver, url)
    print(res)
    total.append(res)
        
driver.close()

{'title': '보안 기술 엔지니어', 'company': '아인시스에이아이랩', 'qualification': '• IDS/IPS 사고분석 경력을 1년 이상 보유하신 분', 'preferential_treatments': '• 전산학, 컴퓨터공학을 전공하신 분 \n• 보안 장비 프로젝트를 경험해보신 분\n• 보안/네트워크 관련 자격증을 소지하신 분'}
{'title': 'Backend 개발팀 시니어 개발자 (6년 이상)', 'company': '모던라이언', 'qualification': '• 6년 이상의 백엔드 개발 경력이 있거나 그에 준하는 실력을 갖추신 분\n• JavaScript / TypeScript 를 이용한 개발에 능숙하신 분\n• 웹 서비스에 대한 전반적인 이해를 갖추신 분\n• RDB(Postgres) 및 ORM에 대한 전반적인 이해를 갖추신 분\n• 도커, 쿠버네티스 등 컨테이너 기반의 서비스에 대한 경험이 있으신 분\n• 1년 이상의 NestJS 실무 프로젝트 개발 경험이 있으신 분\n• 모듈화, 테스트, 유지보수가 용이한 코드에 대한 이해를 갖추고 계신 분\n• 기술 중심이 아닌 유저 중심적인 태도를 가지고 계신 분\n• 배움에 대한 열정을 갖고, 지속적이고 효과적으로 피드백을 주고 받는 분', 'preferential_treatments': '• 하나 이상의 백엔드 서비스나 플랫폼을 직접 설계 및 개발한 경험이 있는 분\n• NestJS, TypeORM, Ethers 등의 라이브러리 사용 경험이 있는 분\n• 사용자 인증과 인가 방식에 대한 설계 및 구현 경험이 있는 분\n• 블록체인과 NFT 도메인에 대한 이해가 있으며, 관련 개발에 경험이 있으신 분\n• 피그마로 제작된 UI / UX 기반으로 기획자와 협업 경험이 있으신 분'}
{'title': '솔루션 운영 엔지니어 (국내)', 'company': '스콥정보통신', 'qualification': '• 학력 : 초대졸 이상\n• 전공 : 무관\n• 경력 : 5년 이상\n\n• 

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.106)
Stacktrace:
0   chromedriver                        0x00000001063c86b8 chromedriver + 4937400
1   chromedriver                        0x00000001063bfb73 chromedriver + 4901747
2   chromedriver                        0x0000000105f7d616 chromedriver + 435734
3   chromedriver                        0x0000000105f63bc1 chromedriver + 330689
4   chromedriver                        0x0000000105f63923 chromedriver + 330019
5   chromedriver                        0x0000000105f7f262 chromedriver + 442978
6   chromedriver                        0x0000000105fff56e chromedriver + 968046
7   chromedriver                        0x0000000105fe4de3 chromedriver + 859619
8   chromedriver                        0x0000000105fb2d7f chromedriver + 654719
9   chromedriver                        0x0000000105fb40de chromed

In [98]:
print(total)

[{'title': '보안 기술 엔지니어', 'company': '아인시스에이아이랩', 'qualification': '• IDS/IPS 사고분석 경력을 1년 이상 보유하신 분', 'preferential_treatments': '• 전산학, 컴퓨터공학을 전공하신 분 \n• 보안 장비 프로젝트를 경험해보신 분\n• 보안/네트워크 관련 자격증을 소지하신 분'}, {'title': 'Backend 개발팀 시니어 개발자 (6년 이상)', 'company': '모던라이언', 'qualification': '• 6년 이상의 백엔드 개발 경력이 있거나 그에 준하는 실력을 갖추신 분\n• JavaScript / TypeScript 를 이용한 개발에 능숙하신 분\n• 웹 서비스에 대한 전반적인 이해를 갖추신 분\n• RDB(Postgres) 및 ORM에 대한 전반적인 이해를 갖추신 분\n• 도커, 쿠버네티스 등 컨테이너 기반의 서비스에 대한 경험이 있으신 분\n• 1년 이상의 NestJS 실무 프로젝트 개발 경험이 있으신 분\n• 모듈화, 테스트, 유지보수가 용이한 코드에 대한 이해를 갖추고 계신 분\n• 기술 중심이 아닌 유저 중심적인 태도를 가지고 계신 분\n• 배움에 대한 열정을 갖고, 지속적이고 효과적으로 피드백을 주고 받는 분', 'preferential_treatments': '• 하나 이상의 백엔드 서비스나 플랫폼을 직접 설계 및 개발한 경험이 있는 분\n• NestJS, TypeORM, Ethers 등의 라이브러리 사용 경험이 있는 분\n• 사용자 인증과 인가 방식에 대한 설계 및 구현 경험이 있는 분\n• 블록체인과 NFT 도메인에 대한 이해가 있으며, 관련 개발에 경험이 있으신 분\n• 피그마로 제작된 UI / UX 기반으로 기획자와 협업 경험이 있으신 분'}, {'title': '솔루션 운영 엔지니어 (국내)', 'company': '스콥정보통신', 'qualification': '• 학력 : 초대졸 이상\n• 전공 : 무관\n• 경력 : 5년 이상\n\

In [89]:
import re

# [word].[word] 형식의 단어 처리
cleaned_list = []
for word in word_list:
    if re.match(r'^\w+\.\w+$', word):
        combined_word = word.replace('.', '')
        cleaned_list.append(combined_word)
    else:
        cleaned_list.extend(re.findall(r'\b\w+\b', word))

print(cleaned_list)

['Web3', 'js', '사용', '경험', '블록체인', '애플리케이션', '및', 'dApp에', '대한', '기본적인', '이해', '오픈', '소스', '프로젝트에', '기여한', '경험']


In [99]:
import csv

with open('//Users/jhnam/workspace/dummy/wanted_crawling_data.csv', 'w', encoding='UTF-8') as f:
    # csv 파일의 헤더 설정
    header = ['title', 'company', 'qualifications', 'preferred', 'skills']

    # 헤더 적용
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    
    # 헤더에 맞게 딕셔너리 값 입력
    for jd in total:
        writer.writerow(jd)